This notebook will be used for the capstone project.

In [9]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
#import geocoder
import csv

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #Url of Wikipedia page with Toronto neighborhoods and postal codes
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table') #Find tables in the html
neighborhood_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
for row in tables[0].tbody.find_all("tr"):
    for col in row.find_all("td"):
        if col.i == None:
            postalCode = col.b.text
            boroughAndNeighborhood = re.split("\(",col.span.text)
            borough = boroughAndNeighborhood[0]
            neighborhoods = boroughAndNeighborhood[1].strip(")")
            neighborhood = neighborhoods.replace(" /",",").replace(")"," ")
            #neighborhood1=(((((col.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            #print(neighborhood1)
            neighborhood_data = neighborhood_data.append({"PostalCode":postalCode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)
        
neighborhood_data['Borough']=neighborhood_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

##############This part of the code reads the csv file and creates an array "rows" that contains the Postal Codes, Latitudes and Longitudes
filename = "Geospatial_Coordinates.csv"
# initializing the titles and rows list
fields = []
rows = []
# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
      
    # extracting field names through first row
    fields = next(csvreader)
  
    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)

#############################This part of the code adds the latitudes and longitudes to the dataframe        
neighborhood_data['Latitude'] = "" #Initialize Latitude column
neighborhood_data['Longitude'] = "" #Initialize Longitude column
for ind in neighborhood_data.index:
    for i in range(len(rows)):
        if neighborhood_data['PostalCode'][ind] == rows[i][0]:
            neighborhood_data['Latitude'][ind] = rows[i][1]
            neighborhood_data['Longitude'][ind] = rows[i][2]


In [13]:
neighborhood_data.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.4647633
4,M7A,Queen's Park,Ontario Provincial Government,43.6623015,-79.3894938
5,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
6,M1B,Scarborough,"Malvern, Rouge",43.8066863,-79.1943534
7,M3B,North York,Don Mills North,43.7459058,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063972,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6571618,-79.3789371
